In [30]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.tools import Tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.agents import initialize_agent, AgentType
import requests

print("Imports completados")

Imports completados


In [31]:
CHROMA_DIR = "chroma_db"
GLOBAL_LLM = None

print("Configuración lista")

Configuración lista


In [32]:
def search_pdfs(query: str) -> str:
    embeddings = OllamaEmbeddings(model="embeddinggemma:300m")
    vectordb = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)
    docs = vectordb.similarity_search(query, k=4)

    if not docs:
        return "No se encontró información relevante en los documentos."

    context = ""
    sources = []
    for d in docs:
        source = d.metadata.get("source_file", "Desconocido")
        page = d.metadata.get("page", "N/A")
        sources.append(f"{source} (pág. {page})")
        context += d.page_content + "\n"

    return f"""Información encontrada en los documentos:

{context}

Fuentes: {', '.join(set(sources))}
"""

pdf_tool = Tool(
    name="Buscar_en_PDFs",
    func=search_pdfs,
    description="Usa esta herramienta para buscar información dentro de documentos PDF indexados."
)

print("Tool de PDFs creado")

Tool de PDFs creado


In [33]:
wikipedia_wrapper = WikipediaAPIWrapper(
    lang="es",
    top_k_results=2,
    doc_content_chars_max=2000
)
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_wrapper)
wikipedia_tool.name = "Buscar_en_Wikipedia"
wikipedia_tool.description = "Usa esta herramienta para buscar información general en Wikipedia en español."

print("Tool de Wikipedia creado")

Tool de Wikipedia creado


In [34]:
def consultar_clima(ciudad: str) -> str:
    """Consulta el clima actual usando la API 2.5 gratuita de OpenWeatherMap"""
    try:
        api_key = "Your_APi"
        ciudad = ciudad.strip()
        
        # Paso 1: Geocoding
        geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={ciudad}&limit=1&appid={api_key}"
        geo_response = requests.get(geo_url, timeout=10)
        
        if geo_response.status_code != 200:
            return f"Error de conexion al buscar {ciudad}. Codigo: {geo_response.status_code}"
        
        geo_data = geo_response.json()
        if not geo_data:
            return f"No se encontro la ciudad {ciudad}. Intenta con otro nombre."
        
        lat = geo_data[0]['lat']
        lon = geo_data[0]['lon']
        nombre_ciudad = geo_data[0].get('local_names', {}).get('es', geo_data[0]['name'])
        
        # Paso 2: Clima actual
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&units=metric&lang=es&appid={api_key}"
        weather_response = requests.get(weather_url, timeout=10)
        
        if weather_response.status_code == 200:
            data = weather_response.json()
            
            temp = data['main']['temp']
            sensacion = data['main']['feels_like']
            descripcion = data['weather'][0]['description']
            humedad = data['main']['humidity']
            viento = data['wind']['speed']
            presion = data['main']['pressure']
            
            # Paso 3: Pronóstico
            forecast_url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&units=metric&lang=es&appid={api_key}"
            forecast_response = requests.get(forecast_url, timeout=10)
            
            pronostico_texto = ""
            if forecast_response.status_code == 200:
                forecast_data = forecast_response.json()
                if 'list' in forecast_data and len(forecast_data['list']) > 0:
                    proxima = forecast_data['list'][0]
                    temp_prox = proxima['main']['temp']
                    desc_prox = proxima['weather'][0]['description']
                    pronostico_texto = f", pronostico proximas horas: {temp_prox}C con {desc_prox}"
            
            return f"Clima en {nombre_ciudad}: temperatura {temp}C (sensacion {sensacion}C), {descripcion}, humedad {humedad}%, viento {viento} m/s, presion {presion} hPa{pronostico_texto}"
        else:
            return f"Error al obtener clima. Codigo HTTP: {weather_response.status_code}"
            
    except Exception as e:
        return f"Error inesperado: {str(e)}"

clima_tool = Tool(
    name="Consultar_Clima",
    func=consultar_clima,
    description="Usa esta herramienta para consultar el clima actual de cualquier ciudad."
)

print("Tool de Clima creado")


Tool de Clima creado


In [37]:
def build_agent():
    global GLOBAL_LLM

    GLOBAL_LLM = ChatOllama(
        model="llama3",
        temperature=0.1
    )

    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )

    prefix = """Eres un asistente útil que SIEMPRE responde en ESPAÑOL.

INSTRUCCIONES IMPORTANTES:
1. NUNCA respondas en inglés, solo en español
2. Cuando el usuario pregunte sobre documentos, DEBES usar la herramienta "Buscar_en_PDFs"
3. Para información general, usa "Buscar_en_Wikipedia"
4. Para el clima, usa "Consultar_Clima"
5. Si la herramienta devuelve información, basa tu respuesta en esa información
6. Si no hay información en los documentos, dilo claramente
7. Todas tus respuestas deben ser en español, sin excepción

Tienes acceso a las siguientes herramientas:"""

    suffix = """¡Comencemos!

Historial de conversación:
{chat_history}

Pregunta del usuario: {input}
{agent_scratchpad}

Recuerda: Tu respuesta DEBE estar COMPLETAMENTE en español."""

    agent = initialize_agent(
        tools=[pdf_tool, wikipedia_tool, clima_tool],
        llm=GLOBAL_LLM,
        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=5,
        early_stopping_method="generate",
        agent_kwargs={
            "prefix": prefix,
            "suffix": suffix
        }
    )
    return agent

# Crear el agente
agent = build_agent()
print("Agente creado y listo para usar")

Agente creado y listo para usar


In [38]:
def forzar_espanol(texto: str) -> str:
    """Traduce cualquier respuesta al español si detecta inglés"""
    global GLOBAL_LLM

    palabras_ingles = [
        "the", "and", "is", "are", "of", "to", "in", "that", 
        "for", "with", "on", "at", "this", "from", "by", "was", "were"
    ]
    
    texto_lower = texto.lower()
    tiene_ingles = any(f" {palabra} " in f" {texto_lower} " for palabra in palabras_ingles)
    
    if not tiene_ingles:
        return texto
    
    prompt = f"""Traduce el siguiente texto al español. Si ya está en español, déjalo igual.
IMPORTANTE: Tu respuesta debe contener SOLO el texto traducido, nada más.

Texto:
{texto}

Traducción al español:"""
    
    try:
        resp = GLOBAL_LLM.invoke(prompt)
        resultado = resp.content if hasattr(resp, "content") else str(resp)
        return resultado.strip()
    except:
        return texto

print("Post-procesador listo")  

Post-procesador listo


In [39]:
def preguntar(pregunta: str):
    """
    Función principal para hacer preguntas al asistente
    
    Ejemplos:
    - preguntar("¿Qué dice el documento sobre inteligencia artificial?")
    - preguntar("¿Quién fue Albert Einstein?")
    - preguntar("¿Cómo está el clima en Madrid?")
    """
    print(f"\n{'='*60}")
    print(f"PREGUNTA: {pregunta}")
    print(f"{'='*60}\n")
    
    try:
        raw = agent.run(pregunta)
        respuesta = forzar_espanol(raw)
        
        print(f"\n{'='*60}")
        print(f"RESPUESTA:")
        print(f"{'='*60}")
        print(respuesta)
        print(f"\n{'='*60}\n")
        
        return respuesta
    except Exception as e:
        error_msg = f" Error al procesar la consulta: {str(e)}"
        print(error_msg)
        return error_msg

print("Función preguntar() lista para usar")
print("\n" + "="*60)
print("SISTEMA LISTO - Usa la siguiente celda para hacer preguntas")
print("="*60)


Función preguntar() lista para usar

SISTEMA LISTO - Usa la siguiente celda para hacer preguntas


In [14]:
preguntar("Segun los pdfs, que es la plataforma preciojusto?")


PREGUNTA: Segun los pdfs, que es la plataforma preciojusto?



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Buscar_en_PDFs
Action Input: plataforma preciojusto

C:\Users\moram\AppData\Local\Temp\ipykernel_42428\3708646641.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)



Observation: Información encontrada en los documentos:

Justificación 
La implementación de "PrecioJusto" se justifica por su potencial para transformar la 
experiencia de compra de los consumidores y promover prácticas comerciales más 
transparentes. En un entorno económico donde cada peso ahorrado marca la 
diferencia, contar con herramientas que faciliten la comparación de precios se ha 
convertido en una necesidad fundamental para las familias. 
"PrecioJusto" no solo simplifica el proceso de comparación de precios, sino que 
también introduce un cambio cultural en los hábitos de consumo, promoviendo la 
planificación y el ahorro sistemático. La plataforma se alinea perfectamente con las 
tendencias actuales de consumo inteligente y aprovechamiento de tecnologías para 
mejorar la calidad de vida. 
La relevancia de "PrecioJusto" se fortalece al considerar: 
1. Contexto económico actual: En periodos de inflación y ajustes presupuestarios, 
las familias necesitan herramientas que les 

'La plataforma "PrecioJusto" es un sistema de comparación de precios diseñado para proporcionar a los consumidores una herramienta poderosa que les permita tomar decisiones de compra informadas y optimizar su presupuesto familiar. Según la información encontrada en los documentos, la justificación de esta plataforma se basa en su potencial para transformar la experiencia de compra de los consumidores y promover prácticas comerciales más transparentes.'

In [15]:
preguntar("Quien fue Gustavo Rojas Pinilla?")


PREGUNTA: Quien fue Gustavo Rojas Pinilla?



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Buscar_en_Wikipedia
Action Input: Gustavo Rojas Pinilla
Observation: Page: Gustavo Rojas Pinilla
Summary: Gustavo Rojas Pinilla (Tunja, 12 de marzo de 1900-Melgar, 17 de enero de 1975), fue un militar, ingeniero civil y político colombiano. Fue presidente de Colombia entre el 13 de junio de 1953 y el 10 de mayo de 1957. Tras el golpe de Estado que le daría al titular Laureano Gómez, ocupando de facto la presidencia de Colombia.. 
Durante su carrera política fue apodado Gurropín debido a las primeras sílabas de su nombre completo.​​​​
Fue dictador de Colombia tras el golpe de Estado que le daría al titular Laureano Gómez, ocupando de facto la presidencia de Colombia del 13 de junio de 1953 al 10 de mayo de 1957 mediante una dictadura militar.​ Su mandato se caracterizó por la realización de trabajos de infraestructura,​ como lo son el Aeropuerto Internaciona

'Gustavo Rojas Pinilla fue un militar, ingeniero civil y político colombiano que ocupó la presidencia de Colombia entre 1953 y 1957. Fue dictador de facto durante ese período y se caracterizó por realizar obras de infraestructura y controlar el petróleo en el país. También es conocido por ser el cuarto Presidente ingeniero en la Historia de Colombia.'

In [17]:
preguntar("como esta el clima en Monteria?")


PREGUNTA: como esta el clima en Monteria?



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Sí
Action: Consultar_Clima
Action Input: Montería
Observation: Clima en Perímetro Urbano Montería: temperatura 30.95C (sensacion 37.95C), lluvia ligera, humedad 74%, viento 2.06 m/s, presion 1006 hPa, pronostico proximas horas: 30.95C con lluvia ligera
Thought:Could not parse LLM output: `¡Claro! El clima actual en Montería es como sigue: temperatura de 30.95°C (sensación de 37.95°C), lluvia ligera, humedad del 74%, viento de 2.06 metros por segundo y presión atmosférica de 1006 hectopascals. Además, el pronóstico para las próximas horas indica que la temperatura seguirá siendo de 30.95°C con lluvia ligera.`
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: `¡Claro! El clima actual en Montería es como sigue: temperatura de 30.95°C (sen

'¡Claro! El clima actual en Montería es como sigue: temperatura de 30.95°C (sensación de 37.95°C), lluvia ligera, humedad del 74%, viento de 2.06 metros por segundo y presión atmosférica de 1006 hectopascals. Además, el pronóstico para las próximas horas indica que la temperatura seguirá siendo de 30.95°C con lluvia ligera.'

In [40]:
preguntar("Busca en Wikipedia, quienes crearon python")


PREGUNTA: Busca en Wikipedia, quienes crearon python



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Sí
Action: Buscar_en_Wikipedia
Action Input: Python
Observation: Page: Python
Summary: Python es un lenguaje de alto nivel de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código. Se trata de un lenguaje de programación multiparadigma, ya que soporta parcialmente la orientación a objetos, programación imperativa y, en menor medida, programación funcional. Es un lenguaje interpretado, dinámico, multiplataforma.
Administrado por Python Software Foundation, posee una licencia de código abierto, denominada Python Software Foundation License.​ Python se clasifica constantemente como uno de los lenguajes de programación más populares, siendo ya en 2025 el más popular y además con una amplia diferencia récord histórico de más de 15 puntos porcentuales sobre el siguiente.​



Page: RAFAEL Python
Summary: El Python es un misil aire-air

'El lenguaje de programación Python fue creado por Guido van Rossum, un desarrollador holandés. Fue diseñado para ser fácilmente legible y entenderse rápidamente, lo que lo hace ideal para uso en proyectos de desarrollo web y científico.'